In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
base = '/users/jmy/data/image_sound'
sbjs_num = 15
sbj_list = [25,26,29,30,31,32,33,34,37,38,39,40,41,43,44]
interest_list = ['Digit','Object','Vs','Magnitude','Animacy']
cd_n = len(interest_list)
hcp = nib.load('/users/jmy/data/MNI_reference/hcp_360_3mm_64.nii').get_fdata()
affine = np.array([[   3. ,   -0. ,   -0. ,  -94.5],
       [  -0. ,    3. ,   -0. , -130.5],
       [   0. ,    0. ,    3. ,  -76.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [ ]:
# RDM codes for each condition
dgt_cd = np.zeros((16,16))
dgt_cd[:10,:10] = 1
obj_cd = np.zeros((16,16))
obj_cd[10:,10:] = 1
vs_cd = np.zeros((16,16))
vs_cd[:10,10:] = 1
vs_cd[10:,:10] = 1
mag_cd = np.zeros((16,16))
mag_cd[7:10,1:4] = 1
ani_cd = np.zeros((16,16))
ani_cd[13:,10:13] = 1
cds_list = [dgt_cd,obj_cd,vs_cd,mag_cd,ani_cd]

In [ ]:
def rdm_reord(org):
    new = org.copy()
    new[10:15,10:15] = org[11:,11:]
    new[15,15] = org[10,10]
    new[-1,10:15] = org[10,11:]
    new[10:15,-1] = org[11:,10]
    return new

In [ ]:
vols_off = np.zeros((cd_n,sbjs_num,64,76,64))
il16 = np.tril_indices(16,-1)
for sbj_idx in range(sbjs_num): #25,39
    # load brain rdm
    load_path = base+'/RDMs/Neural/16x16/P'+str(sbj_list[sbj_idx])+'_i.npz'
    l = np.load(load_path,allow_pickle=True)
    brain_rdm = l['rdm']
    brain_inform_list = l['info_list'] 
    # spearman corr
    nvv = brain_rdm.shape[0]
    for cdi,code in enumerate(cds_list):
        vol = np.zeros((64,76,64))
        for vi in range(nvv):
            idx = brain_inform_list[vi][1]
            rdm_ = rdm_reord(brain_rdm[vi])
            r, _ = stats.spearmanr(code[il16],rdm_[il16])
            vol[idx] = r
        vols_off[cdi,sbj_idx] = vol

In [ ]:
# Zero center (for each condition)
for ti,type_ in enumerate(interest_list):
    xlim = max(abs(vols_off[ti].min()),abs(vols_off[ti].max()))
    f,axes = plt.subplots(3,5, figsize=(20,11),constrained_layout=True,facecolor='white')
    f.suptitle(type_, fontsize=17)
    ax = axes.flat
    for i in range(sbjs_num):
        vol = vols_off[ti,i]
        ax[i].set_title('SKKU '+str(sbj_list[i]-25),fontsize=13)
        ax[i].set_xlim([xlim*-1.,xlim])
        ax[i].axvline(x=0.0,color='r',linestyle='--')
        sns.distplot(vol[np.where(vol != 0)],ax=ax[i])
    plt.show()

In [ ]:
# group mask
intsec_masks = np.zeros((cd_n,64,76,64),dtype=int)
for ti,type_ in enumerate(interest_list):
    vol_sum = np.sum((vols_off[ti] != 0), axis=0)
    ints_idx = np.where(vol_sum == sbjs_num)
    print(type_,':',ints_idx[0].shape[0])
    intsec_masks[ti][ints_idx] = 1
# ttest
savebase = base+'/01_STEP/16x16/offd/Ps'+str(sbjs_num)+'_'
os.makedirs(savebase,exist_ok=False)
for ti,type_ in enumerate(interest_list):
    tmpvols_off = vols_off[ti].copy()
    # mask
    msk_idx = np.where(intsec_masks[ti] == 1)
    nvv = msk_idx[0].shape[0]
    # r to z
    vecs = np.arctanh(tmpvols_off.transpose(1,2,3,0)[msk_idx])
    # t-test    
    tmpt = np.zeros((nvv))
    tmpp = np.zeros((nvv))
    for vi, vec in enumerate(vecs):
        tmpt[vi], tmpp[vi] = stats.ttest_1samp(vec,0.)
    # save t
    vol_tmp = np.zeros((64,76,64))
    vol_tmp[msk_idx] = tmpt
    img1 = nib.Nifti1Image(vol_tmp,affine=affine)
    if type_ in ['Magnitude','Animacy']:
        file_key = type_+'_r3_16x16'
    else:
        file_key = type_+'_r3'
    nib.save(img1,savebase+'tmap_'+file_key+'.nii')
    # save p
    vol_tmp = np.zeros((64,76,64))
    vol_tmp[msk_idx] = tmpp
    img1 = nib.Nifti1Image(vol_tmp,affine=affine)
    nib.save(img1,savebase+'pval_'+file_key+'.nii')

# p-value correction

In [ ]:
import os
import scipy
from tqdm import tqdm
from multiprocessing import Pool, Lock

In [ ]:
idx2shf = np.arange(nvv)
sbj_idx_ = np.arange(sbjs_num)

In [ ]:
# using multi core
p_cnt = 25 # number of cores 
run_cnt = int(nvv*10000/p_cnt) 
print(nvv*10000/p_cnt)

In [ ]:
for ti, type_ in enumerate(interest_list):
    tmpvols = vols_off[ti].copy()
    # mask
    msk_idx = np.where(intsec_masks[ti] == 1)
    nvv = msk_idx[0].shape[0]
    # r to z
    vecs = np.arctanh(tmpvols.transpose(1,2,3,0)[msk_idx])
    p_out = []
    boot_ts = np.zeros((1))
    def ttest_Rndperm(self):
        global sbj_idx_, vecs, boot_ts
        scipy.random.seed(int.from_bytes(os.urandom(3), byteorder='little'))
        rnd_idx = np.random.choice(idx2shf,sbjs_num,replace=False)
        boot_ts, _ = stats.ttest_1samp(vecs[rnd_idx,sbj_idx_],0.)
        return boot_ts
    if __name__=='__main__':
        p = Pool(processes = p_cnt) # 25
        for i in tqdm(range(run_cnt), desc='Total Progress') : # 400
            p_out.append(p.map(ttest_Rndperm, range(p_cnt))) # 25
        p.close()
        p.join()
    pooled_ts = np.array(p_out,dtype=np.float16).reshape(p_cnt*run_cnt)
    sns.distplot(pooled_ts)
    plt.title(type_)
    plt.show()
    # save
    if type_ in ['Magnitude','Animacy']:
        file_key = type_+'_r3_16x16'
    else:
        file_key = type_+'_r3'
    np.savez(savebase+'rndperm_tscore_'+file_key+'.npz',pooled_ts=pooled_ts)

## Thresholded tmap

In [ ]:
savebase = base+'/01_STEP/Ps'+str(sbjs_num)+'_'

In [ ]:
pval = 0.05
for ti,type_ in enumerate(interest_list):
    if type_ in ['Magnitude','Animacy']:
        file_key = type_+'_r3_16x16'
    else:
        file_key = type_+'_r3'
    tvol = nib.load(savebase+'tmap_'+file_key+'.nii').get_fdata()
    msk_idx = np.where(tvol!=0)
    print(type_, msk_idx[0].shape[0])
    tmpt = tvol[msk_idx]
    pooled_ts = np.load(savebase+'rndperm_tscore_'+file_key+'.npz')['pooled_ts']
    # threshold
    lb, ub = np.percentile(pooled_ts,[pval*100/2,100-pval*100/2])
    tmpt[np.where((lb < tmpt) & (tmpt< ub))] = 0
    vol_tmp = np.zeros((64,76,64))
    vol_tmp[msk_idx] = tmpt
    img1 = nib.Nifti1Image(vol_tmp,affine=affine)
    savepath = savebase+str(pval)+'_correted_tmap_'+file_key+'.nii'
    nib.save(img1,savepath)